In [1]:
#Importing libraries
import pandas as pd
import numpy as np
from scipy.optimize import fmin_slsqp
import os
from pprint import pprint
import random

#Listing the existing files in the dir
for root, dirs, files in os.walk("."):  
    for filename in files:
        print(filename)

code.ipynb
code_2.ipynb
code_3.ipynb
index.html
README.md
COMMIT_EDITMSG
config
description
FETCH_HEAD
HEAD
index
ORIG_HEAD
applypatch-msg.sample
commit-msg.sample
fsmonitor-watchman.sample
post-update.sample
pre-applypatch.sample
pre-commit.sample
pre-push.sample
pre-rebase.sample
pre-receive.sample
prepare-commit-msg.sample
update.sample
exclude
HEAD
master
master
24780b6fd326cf4b955325067d443e8ff4c949
a18e86d302f98d38c2b0a7f695f432503deb99
085fb5674218edb15c32f80821cacfdecbaf74
af9c2494cb516ecd4069106fb2552b6c56f964
dc2a2224dcf3fb4dc3c520bbe7418c17e17b72
2ea67fa45d80aa174f684e71cbf27fe409e715
2f8eb19024635bdc5b17249e3153ce695b0f3f
258848b6b8486b3d84643f229b467263c0e16c
14d86ed8aca499bd6544284f05304ce272a6ad
2a4eb39e8337081a7e871d17fef54c4c914168
ccd761751157b0447ce7c4367f63cbe7bcc2c2
55d56c9f372fa638fb3b145deb48fb59c029af
5d4b2cdd2409becd8b9e699da03545d62c92ce
b4b65366cac7f51d24ace5447049ace2f974cf
30e325be5f9f1374b2697404c320e8282c8df7
b092a6134826e550dba7277253e6dd0b0d9fd5
dcb7269

In [109]:
#Loading df
df = pd.read_csv('./files/data-2018-09-24.csv') #skiprows = 1)
df = df.drop(df.loc[:,['name', 'pft']].head(0).columns, axis=1)
df.head()

,firm,x1,x2,x3,x4,x5,y1,y2,y3
0,A,86.13,16.24,48.21,49.69,9,54.53,58.98,38.16
1,B,29.26,10.24,41.96,40.65,5,24.69,33.89,26.02
2,C,43.12,11.31,38.19,35.03,9,36.41,40.62,28.51
3,D,24.96,6.14,24.81,25.15,7,14.94,17.58,16.19
4,E,11.62,2.21,6.85,6.37,4,7.81,6.94,5.37


In [110]:
inpt_df = df.iloc[:, 1:6]
inpt_arr = np.array(inpt_df)
outpt_df = df.iloc[:, 6:]
outpt_arr = np.array(outpt_df)
comp = np.array(df.iloc[:, 0])
print(inpt_arr, outpt_arr, comp)

[[86.13 16.24 48.21 49.69  9.  ]
 [29.26 10.24 41.96 40.65  5.  ]
 [43.12 11.31 38.19 35.03  9.  ]
 [24.96  6.14 24.81 25.15  7.  ]
 [11.62  2.21  6.85  6.37  4.  ]
 [11.88  4.97 18.73 18.04  4.  ]
 [32.64  6.88 28.1  25.45  7.  ]
 [20.79 12.97 54.85 52.07  8.  ]
 [34.4  11.04 38.16 42.4   8.  ]
 [61.74 14.5  49.09 42.92  9.  ]
 [52.92 11.67 39.48 39.64  5.  ]
 [36.   10.15 37.8  39.52  5.  ]
 [39.2  10.8  41.04 41.12  7.  ]
 [14.6   2.88  9.64 11.14  3.  ]
 [ 4.29  5.42 21.45 17.27  5.  ]
 [27.25 14.17 56.46 55.26  9.  ]
 [22.63  4.43 15.4  15.    2.  ]
 [28.    7.61 28.73 27.04  9.  ]
 [53.56 13.7  53.04 49.85  7.  ]
 [25.42  9.05 29.69 31.74  4.  ]
 [31.57 10.08 39.34 40.57  6.  ]
 [16.34  5.84 20.89 22.1   4.  ]
 [44.28 14.14 56.7  52.27 11.  ]
 [19.74  6.43 24.2  25.66  3.  ]
 [24.4   8.05 33.42 31.29  7.  ]
 [41.4  11.7  44.01 46.35  7.  ]] [[54.53 58.98 38.16]
 [24.69 33.89 26.02]
 [36.41 40.62 28.51]
 [14.94 17.58 16.19]
 [ 7.81  6.94  5.37]
 [12.59 16.85 12.84]
 [17.06 16.99 1

In [111]:
class DEA(object):
    random.seed(5)
    def __init__(self, inputs, outputs):
        """
        Initialize the DEA object with input data
        n = number of entities (observations)
        m = number of inputs (variables, features)
        r = number of outputs
        :param inputs: inputs, n x m numpy array
        :param outputs: outputs, n x r numpy array
        :return: self
        """

        # supplied data
        self.inputs = inputs
        self.outputs = outputs

        # parameters
        self.n = inputs.shape[0]
        self.m = inputs.shape[1]
        self.r = outputs.shape[1]

        # iterators
        self.unit_ = range(self.n)
        self.input_ = range(self.m)
        self.output_ = range(self.r)

        # result arrays
        self.output_w = np.zeros((self.r, 1), dtype=np.float)  # output weights
        self.input_w = np.zeros((self.m, 1), dtype=np.float)  # input weights
        self.lambdas = np.zeros((self.n, 1), dtype=np.float)  # unit efficiencies
        self.efficiency = np.zeros_like(self.lambdas)  # thetas

        # names
        self.names = []

    def __efficiency(self, unit):
        """
        Efficiency function with already computed weights
        :param unit: which unit to compute for
        :return: efficiency
        """

        # compute efficiency
        denominator = np.dot(self.inputs, self.input_w)
        numerator = np.dot(self.outputs, self.output_w)

        return (numerator/denominator)[unit]

    def __target(self, x, unit):
        """
        Theta target function for one unit
        :param x: combined weights
        :param unit: which production unit to compute
        :return: theta
        """
        in_w, out_w, lambdas = x[:self.m], x[self.m:(self.m+self.r)], x[(self.m+self.r):]  # unroll the weights
        denominator = np.dot(self.inputs[unit], in_w)
        numerator = np.dot(self.outputs[unit], out_w)

        return numerator/denominator

    def __constraints(self, x, unit):
        """
        Constraints for optimization for one unit
        :param x: combined weights
        :param unit: which production unit to compute
        :return: array of constraints
        """

        in_w, out_w, lambdas = x[:self.m], x[self.m:(self.m+self.r)], x[(self.m+self.r):]  # unroll the weights
        constr = []  # init the constraint array

        # for each input, lambdas with inputs
        for input in self.input_:
            t = self.__target(x, unit)
            lhs = np.dot(self.inputs[:, input], lambdas)
            cons = t*self.inputs[unit, input] - lhs
            constr.append(cons)

        # for each output, lambdas with outputs
        for output in self.output_:
            lhs = np.dot(self.outputs[:, output], lambdas)
            cons = lhs - self.outputs[unit, output]
            constr.append(cons)

        # for each unit
        for u in self.unit_:
            constr.append(lambdas[u])

        return np.array(constr)

    def __optimize(self):
        """
        Optimization of the DEA model
        Use: http://docs.scipy.org/doc/scipy-0.17.0/reference/generated/scipy.optimize.linprog.html
        A = coefficients in the constraints
        b = rhs of constraints
        c = coefficients of the target function
        :return:
        """
        d0 = self.m + self.r + self.n
        # iterate over units
        for unit in self.unit_:
            # weights
            x0 = np.random.rand(d0) - 0.5
            x0 = fmin_slsqp(self.__target, x0, f_ieqcons=self.__constraints, args=(unit,))
            # unroll weights
            self.input_w, self.output_w, self.lambdas = x0[:self.m], x0[self.m:(self.m+self.r)], x0[(self.m+self.r):]
            self.efficiency[unit] = self.__efficiency(unit)

    def name_units(self, names):
        """
        Provide names for units for presentation purposes
        :param names: a list of names, equal in length to the number of units
        :return: nothing
        """

        assert(self.n == len(names))

        self.names = names

    def fit(self):
        global df_results, alls
        """
        Optimize the dataset, generate basic table
        :return: table
        """

        self.__optimize()  # optimize
        print("---------------------------\n")       

        efficient = {}
        not_efficient = {} 
        alls = {}
        for n, eff in enumerate(self.efficiency):           
            if eff >= 1.:
                efficient.update({self.names[n]: eff[0]}) 
            else:
                not_efficient.update({self.names[n]: eff[0]})
        for n, eff in enumerate(self.efficiency):
            alls.update({self.names[n]: eff[0]})
        print("Efficient units:")     
        print(str(efficient).replace("{","").replace("}", ""))       
        print("\n")    
        print("Inefficient units:")
        print(str(not_efficient).replace("{","").replace("}", ""))  
        
def save_results(dataframe):  
    df_results = pd.DataFrame([]) 
    df_results = df_results.append(alls, ignore_index = True).T
    df_results = df_results.reset_index()
    df_results.columns = ['firm', 'efficiency']    
    dataframe = dataframe.merge(df_results)
    return dataframe        

dea = DEA(inpt_arr, outpt_arr)
dea.name_units(comp)
dea.fit()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.9999999995109133
            Iterations: 15
            Function evaluations: 547
            Gradient evaluations: 15
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.9663169564274546
            Iterations: 13
            Function evaluations: 468
            Gradient evaluations: 13
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.9958121455834887
            Iterations: 18
            Function evaluations: 649
            Gradient evaluations: 18
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.9321402844295841
            Iterations: 35
            Function evaluations: 1253
            Gradient evaluations: 34
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.0000001166477015
            Iterations: 8
            Fun

In [112]:
save_results(df).head()

,firm,x1,x2,x3,x4,x5,y1,y2,y3,efficiency
0,A,86.13,16.24,48.21,49.69,9,54.53,58.98,38.16,1.000000
1,B,29.26,10.24,41.96,40.65,5,24.69,33.89,26.02,0.966317
2,C,43.12,11.31,38.19,35.03,9,36.41,40.62,28.51,0.995812
3,D,24.96,6.14,24.81,25.15,7,14.94,17.58,16.19,0.932140
4,E,11.62,2.21,6.85,6.37,4,7.81,6.94,5.37,1.000000
